In [15]:
!pip install ecmwf-opendata

  Using cached ecmwf_opendata-0.3.23-py3-none-any.whl.metadata (22 kB)
  Using cached multiurl-0.3.7-py3-none-any.whl.metadata (2.8 kB)
     ---------------------------------------- 0.0/57.7 kB ? eta -:--:--
     ------------- ------------------------ 20.5/57.7 kB 682.7 kB/s eta 0:00:01
     ---------------------------------------- 57.7/57.7 kB 1.0 MB/s eta 0:00:00
Using cached ecmwf_opendata-0.3.23-py3-none-any.whl (20 kB)
Using cached multiurl-0.3.7-py3-none-any.whl (21 kB)
   ---------------------------------------- 0.0/78.5 kB ? eta -:--:--
   ---------------------------------------- 78.5/78.5 kB 2.2 MB/s eta 0:00:00


In [18]:
!pip install xarray cfgrib

  Using cached cfgrib-0.9.15.0-py3-none-any.whl.metadata (55 kB)
  Using cached findlibs-0.1.2-py3-none-any.whl.metadata (4.5 kB)
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   -- ------------------------------------- 0.1/1.3 MB 3.8 MB/s eta 0:00:01
   ---------- ----------------------------- 0.3/1.3 MB 5.4 MB/s eta 0:00:01
   -------------------------- ------------- 0.9/1.3 MB 7.7 MB/s eta 0:00:01
   ---------------------------------------- 1.3/1.3 MB 8.3 MB/s eta 0:00:00
Using cached cfgrib-0.9.15.0-py3-none-any.whl (48 kB)
   ---------------------------------------- 0.0/63.8 kB ? eta -:--:--
   ---------------------------------------- 63.8/63.8 kB 3.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/7.0 MB ? eta -:--:--
   -- ------------------------------------- 0.5/7.0 MB 16.0 MB/s eta 0:00:01
   ------ --------------------------------- 1.1/7.0 MB 13.4 MB/s eta 0:00:01
   ------- -------------------------------- 1.4/7.0 MB 10.9 MB/s eta 0:0

In [17]:
# Import the client library
from ecmwf.opendata import Client

# Initialize the client.
# You can specify the source: 'ecmwf' (default), 'aws', or 'azure'.[2]
client = Client(source="ecmwf")

request = {
    "step": "24",             # Forecast hour (e.g., 24 hours ahead)
    "stream": "oper",         # Forecasting system: 'oper' for high-resolution
    "type": "fc",             # Type of data: 'fc' for forecast
    "param": ["2t", "10u", "10v"], # Parameter short names: 2m temperature, 10m U-wind, 10m V-wind
}

# Execute the retrieve method to download the data
try:
    # The retrieve method can also return the date/time of the data it downloaded
    result = client.retrieve(request, "latest_forecast_data.grib2")
    print(f"Download complete for forecast run: {result.date}")
    print("Data saved to latest_forecast_data.grib2")

except Exception as e:
    print(f"An error occurred during download: {e}")

By downloading data from the ECMWF open data dataset, you agree to the terms: Attribution 4.0 International (CC BY 4.0). Please attribute ECMWF when downloading this data.
An error occurred during download: 'Result' object has no attribute 'date'


In [20]:
# Import the xarray library
import xarray as xr

# Define the path to your downloaded GRIB file
grib_file_path = 'latest_forecast_data.grib2'

# Open the GRIB file using xarray with the 'cfgrib' engine [3]
# This loads the data into a Dataset object.
try:
    ds = xr.open_dataset(grib_file_path, engine='cfgrib')

    # Print the dataset to see its structure, variables, and coordinates
    print("Successfully loaded GRIB file into an xarray.Dataset:")
    print(ds)

    # You can now access individual variables like this:
    # temperature_data = ds['t2m']
    # print("\n2m Temperature DataArray:")
    # print(temperature_data)

except Exception as e:
    print(f"An error occurred: {e}")
    print("Please ensure that the 'eccodes' library is correctly installed on your system.")


skipping variable: paramId==167 shortName='t2m'
Traceback (most recent call last):
  File "c:\Programming\envs\py310\lib\site-packages\cfgrib\dataset.py", line 725, in build_dataset_components
    dict_merge(variables, coord_vars)
  File "c:\Programming\envs\py310\lib\site-packages\cfgrib\dataset.py", line 641, in dict_merge
    raise DatasetBuildError(
cfgrib.dataset.DatasetBuildError: key present and new value is different: key='heightAboveGround' value=Variable(dimensions=(), data=10.0) new_value=Variable(dimensions=(), data=2.0)


Successfully loaded GRIB file into an xarray.Dataset:
<xarray.Dataset> Size: 8MB
Dimensions:            (latitude: 721, longitude: 1440)
Coordinates:
    time               datetime64[ns] 8B ...
    step               timedelta64[ns] 8B ...
    heightAboveGround  float64 8B ...
  * latitude           (latitude) float64 6kB 90.0 89.75 89.5 ... -89.75 -90.0
  * longitude          (longitude) float64 12kB -180.0 -179.8 ... 179.5 179.8
    valid_time         datetime64[ns] 8B ...
Data variables:
    u10                (latitude, longitude) float32 4MB ...
    v10                (latitude, longitude) float32 4MB ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-09-17T14:33 GRIB to CDM+CF via cfgrib-0.9.1

c:\Programming\envs\py310\lib\site-packages\cfgrib\xarray_plugin.py:131: FutureWarning: In a future version, xarray will not decode timedelta values based on the presence of a timedelta-like units attribute by default. Instead it will rely on the presence of a timedelta64 dtype attribute, which is now xarray's default way of encoding timedelta64 values. To continue decoding timedeltas based on the presence of a timedelta-like units attribute, users will need to explicitly opt-in by passing True or CFTimedeltaCoder(decode_via_units=True) to decode_timedelta. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(


In [23]:
df = ds.to_dataframe()
df.reset_index(inplace=True)

# Display the first few rows of the DataFrame to see the structure
print("\nData structure (first 5 rows):")
df


Data structure (first 5 rows):


,latitude,longitude,time,step,heightAboveGround,valid_time,u10,v10
0,90.0,-180.00,2025-09-16 18:00:00,1 days,10.0,2025-09-17 18:00:00,1.684219,6.171692
1,90.0,-179.75,2025-09-16 18:00:00,1 days,10.0,2025-09-17 18:00:00,1.684219,6.171692
2,90.0,-179.50,2025-09-16 18:00:00,1 days,10.0,2025-09-17 18:00:00,1.684219,6.171692
3,90.0,-179.25,2025-09-16 18:00:00,1 days,10.0,2025-09-17 18:00:00,1.684219,6.171692
4,90.0,-179.00,2025-09-16 18:00:00,1 days,10.0,2025-09-17 18:00:00,1.684219,6.171692
...,...,...,...,...,...,...,...,...
1038235,-90.0,178.75,2025-09-16 18:00:00,1 days,10.0,2025-09-17 18:00:00,-5.190781,1.265442
1038236,-90.0,179.00,2025-09-16 18:00:00,1 days,10.0,2025-09-17 18:00:00,-5.190781,1.265442
1038237,-90.0,179.25,2025-09-16 18:00:00,1 days,10.0,2025-09-17 18:00:00,-5.190781,1.265442
1038238,-90.0,179.50,2025-09-16 18:00:00,1 days,10.0,2025-09-17 18:00:00,-5.190781,1.265442


In [1]:
import pandas as pd
import numpy as np

# Data Statistic

In [13]:
df = pd.read_csv('household_power_consumption.csv')
df

,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,16/12/2006,17:24:00,4.216,0.418,234.84,18.4,0,1,17.0
1,16/12/2006,17:25:00,5.36,0.436,233.63,23,0,1,16.0
2,16/12/2006,17:26:00,5.374,0.498,233.29,23,0,2,17.0
3,16/12/2006,17:27:00,5.388,0.502,233.74,23,0,1,17.0
4,16/12/2006,17:28:00,3.666,0.528,235.68,15.8,0,1,17.0
...,...,...,...,...,...,...,...,...,...
1048570,13/12/2008,21:34:00,0.426,0.076,242.27,1.8,0,0,0.0
1048571,13/12/2008,21:35:00,0.424,0.076,242.1,1.8,0,0,0.0
1048572,13/12/2008,21:36:00,0.422,0.076,241.73,1.8,0,0,0.0
1048573,13/12/2008,21:37:00,0.422,0.078,242.56,1.8,0,0,0.0


In [4]:
df['Country'].value_counts()

Country
USA          1053
UK           1034
Australia    1021
Canada       1017
India         996
Japan         994
China         985
Germany       971
Russia        967
Brazil        962
Name: count, dtype: int64

In [5]:
df.describe()

,Year,Total Energy Consumption (TWh),Per Capita Energy Use (kWh),Renewable Energy Share (%),Fossil Fuel Dependency (%),Industrial Energy Use (%),Household Energy Use (%),Carbon Emissions (Million Tons),Energy Price Index (USD/kWh)
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,2012.151400,5142.564425,25039.950516,47.322925,44.932851,40.057585,25.043290,2536.148007,0.273466
std,7.155236,2848.754040,14205.659284,24.603767,20.202800,11.538756,8.597024,1424.105067,0.130782
min,2000.000000,100.480000,500.270000,5.000000,10.010000,20.000000,10.000000,50.640000,0.050000
25%,2006.000000,2713.882500,12683.220000,26.110000,27.337500,30.217500,17.610000,1293.330000,0.160000
50%,2012.000000,5190.850000,25098.770000,47.150000,45.110000,39.980000,25.090000,2568.015000,0.270000
75%,2018.000000,7579.977500,37113.282500,68.682500,62.430000,50.150000,32.570000,3766.182500,0.390000
max,2024.000000,9999.260000,49989.570000,90.000000,80.000000,60.000000,40.000000,4999.340000,0.500000


In [12]:
df[df['Country'] == 'USA']

,Country,Year,Total Energy Consumption (TWh),Per Capita Energy Use (kWh),Renewable Energy Share (%),Fossil Fuel Dependency (%),Industrial Energy Use (%),Household Energy Use (%),Carbon Emissions (Million Tons),Energy Price Index (USD/kWh)
21,USA,2021,5902.59,42979.05,22.22,39.04,55.11,23.68,2353.09,0.33
25,USA,2020,8381.55,39081.82,23.53,20.21,55.39,26.79,3412.93,0.25
51,USA,2015,8018.31,15314.01,53.60,18.51,35.44,18.73,2264.28,0.28
78,USA,2019,6586.80,2821.87,45.46,24.59,24.44,14.84,2028.48,0.43
83,USA,2013,8328.28,7382.19,76.47,23.64,25.66,38.24,1679.70,0.23
...,...,...,...,...,...,...,...,...,...,...
9952,USA,2024,4427.15,1846.81,46.38,16.13,50.30,23.51,3221.04,0.45
9959,USA,2005,6634.13,23527.73,85.59,43.79,31.67,23.72,4898.29,0.37
9969,USA,2015,2485.76,18910.57,53.43,27.69,28.78,37.17,3269.69,0.44
9970,USA,2018,3220.99,45388.21,14.88,11.72,45.72,26.22,3375.78,0.41


# Data Processing

In [2]:
df = pd.read_csv('global_energy_consumption.csv')
df

,Country,Year,Total Energy Consumption (TWh),Per Capita Energy Use (kWh),Renewable Energy Share (%),Fossil Fuel Dependency (%),Industrial Energy Use (%),Household Energy Use (%),Carbon Emissions (Million Tons),Energy Price Index (USD/kWh)
0,Canada,2018,9525.38,42301.43,13.70,70.47,45.18,19.96,3766.11,0.12
1,Germany,2020,7922.08,36601.38,33.63,41.95,34.32,22.27,2713.12,0.08
2,Russia,2002,6630.01,41670.20,10.82,39.32,53.66,26.44,885.98,0.26
3,Brazil,2010,8580.19,10969.58,73.24,16.71,30.55,27.60,1144.11,0.47
4,Canada,2006,848.88,32190.85,73.60,74.86,42.39,23.43,842.39,0.48
...,...,...,...,...,...,...,...,...,...,...
9995,Canada,2017,9786.50,30481.07,14.87,70.25,53.93,19.94,620.85,0.29
9996,UK,2011,7200.88,37402.75,89.00,16.55,24.35,20.57,830.73,0.48
9997,India,2011,2579.82,44516.91,46.65,68.06,57.01,33.73,993.78,0.08
9998,UK,2020,5950.39,22289.39,43.43,68.57,34.94,25.40,199.85,0.47


## Global Data

In [8]:
print("\nAggregating data for global analysis...")
df_global = df.groupby('Year').sum().reset_index()
df_global.rename(columns={'Year': 'ds', 'Total Energy Consumption (TWh)': 'y'}, inplace=True)
df_global['ds'] = pd.to_datetime(df_global['ds'], format='%Y')
df_global.set_index('ds', inplace=True)

print("\nGlobal Aggregated Data:")
df_global


Aggregating data for global analysis...

Global Aggregated Data:


,Country,y,Per Capita Energy Use (kWh),Renewable Energy Share (%),Fossil Fuel Dependency (%),Industrial Energy Use (%),Household Energy Use (%),Carbon Emissions (Million Tons),Energy Price Index (USD/kWh)
ds,,,,,,,,,
2000-01-01,AustraliaBrazilUKGermanyJapanUSARussiaUSAIndia...,1818935.94,9618051.08,18446.44,17159.24,15076.89,9448.78,935111.45,107.87
2001-01-01,CanadaIndiaBrazilRussiaChinaCanadaRussiaUSACan...,1974854.60,10362489.37,19354.08,18045.54,15964.86,9631.68,973735.34,105.59
2002-01-01,RussiaBrazilBrazilIndiaUKJapanRussiaBrazilCana...,1908410.61,9328022.02,17027.25,16547.89,14650.11,9452.65,915015.85,102.80
2003-01-01,JapanUKChinaGermanyJapanAustraliaChinaBrazilCa...,2092794.51,10348115.67,18978.26,17954.80,15976.05,9927.29,1045909.32,108.53
2004-01-01,AustraliaRussiaIndiaCanadaCanadaRussiaUSAUSACh...,1994830.95,9493200.00,17354.15,16822.08,15166.27,9072.59,938743.12,102.32
2005-01-01,JapanIndiaCanadaCanadaChinaBrazilAustraliaChin...,2024460.81,9682051.00,18300.68,17886.44,15224.12,9857.12,991805.79,112.52
2006-01-01,CanadaRussiaIndiaChinaUSAUSAUSAChinaBrazilUKJa...,1938927.85,9427511.72,19224.02,18353.76,15826.61,10075.36,989940.14,101.41
2007-01-01,ChinaUKRussiaUSACanadaCanadaChinaBrazilIndiaUS...,2164026.58,10829500.06,19402.44,18507.74,16407.37,10240.10,1052348.20,113.22
2008-01-01,CanadaBrazilAustraliaJapanCanadaCanadaIndiaGer...,1996911.00,9511201.38,17493.71,16579.50,15647.02,9670.77,974748.77,104.60
